So basically in this code, we are multiplying two polynomials using Fast Fourier Transform (FFT). Normally if you just multiply polynomials term-by-term, it takes a lot of time like O(n^2). But FFT lets us do it super fast in O(nlogn)

1.First, we pad the polynomials with zeros so they become the right size (IN FFT we take a power of 2).

2.Then we use FFT to convert the polynomials from coefficient form into frequency form (like how signals are transformed).

3.Once both are in frequency domain, we just multiply point by point — it's way faster than distributing terms manually.

4.After that, we use inverse FFT to convert it back to normal polynomial (time domain).

5.Finally, we round the results because FFT gives some tiny floating point errors.

In [22]:
import numpy as np

In [24]:

# so like this function gives next power of 2 which is ≥ x
def next_power_of_two(x):
    n = 1
    while n < x:
        n *= 2  # keep doubling until we pass x
    return n

# here we are just adding zeros at the end to make both lists same size
def pad_poly(poly, target_length):
    return poly + [0] * (target_length - len(poly))

# this is the fft function we use to go from time domain to frequency domain
def recursive_fft(sequence):
    n = len(sequence)
    if n == 1:
        return sequence  # nothing to do for 1 element

    # splitting into even and odd index elements
    even = recursive_fft(sequence[0::2])
    odd = recursive_fft(sequence[1::2])

    # combine the results from both halves
    result = [0] * n
    for k in range(n // 2):
        angle = -2 * np.pi * k / n
        w = np.exp(1j * angle)  # this is like rotation factor
        result[k] = even[k] + w * odd[k]
        result[k + n // 2] = even[k] - w * odd[k]

    return result

# ok now inverse of fft – we flip signs, fft it again, and divide
def recursive_ifft(sequence):
    n = len(sequence)
    # conjugate all values first
    conj_seq = [np.conj(x) for x in sequence]
    fft_res = recursive_fft(conj_seq)
    # now conjugate again and divide by n to finish the inverse
    return [np.conj(x) / n for x in fft_res]

# here’s the main function where the actual poly multiplication happens
def multiply_polynomials_fft(p1, p2):
    degree = max(len(p1), len(p2)) - 1
    total_degree = 2 * degree  # coz when we multiply two degree d polynomials, degree becomes 2d
    n = next_power_of_two(total_degree + 1)  # we need size to be power of 2 ≥ 2d + 1

    # pad both polys so they are of same size and size is power of 2
    p1_padded = pad_poly(p1, n)
    p2_padded = pad_poly(p2, n)

    # convert to complex numbers, coz fft needs that
    p1_complex = [complex(x) for x in p1_padded]
    p2_complex = [complex(x) for x in p2_padded]

    # now fft both polynomials
    fft_p1 = recursive_fft(p1_complex)
    fft_p2 = recursive_fft(p2_complex)

    # multiply each value pointwise in freq domain
    fft_result = [fft_p1[i] * fft_p2[i] for i in range(n)]

    # now bring it back to time domain
    result_coeffs_complex = recursive_ifft(fft_result)

    # round off all the real parts to nearest integer
    result_coeffs = [round(x.real) for x in result_coeffs_complex[:total_degree + 1]]

    return result_coeffs


In [26]:
poly1 = [1, 2, 3]  # this means 1 + 2x + 3x^2
poly2 = [4, 5, 6]  # this means 4 + 5x + 6x^2

# so now we call our function and print result
product = multiply_polynomials_fft(poly1, poly2)
print("Product Coefficients:", product)
# Output should be [4, 13, 28, 27, 18]

Product Coefficients: [4, 13, 28, 27, 18]
